### Tutorial: Optimal Strategy Given a Fixed Observing Budget

In this simplest example, let's say you have a budget of 30 observations on the NEID spectrograph on WIYN spectrograph on Kitt Peak in Arizona, and you want to spend them on a single target. Let's say you also have to plan around already-booked nights on the telescope. Finally, let's say your star is an exact copy of AU Mic. Given these constraints, how do you best allocate your budget to maximize the Fisher Information (or minimize the uncertainty) on the radial velocity semi-amplitude, K?


In [33]:
import numpy as np 
import scipy
print(np.__version__)
print(scipy.__version__)
import pandas as pd
import random
import exoplanet
import astropy 
import pymc3
import pymc3_ext
import celerite2
from numpy.linalg import inv, det, solve, cond
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib

import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
#from jax import random

from gaspery import calculate_fi, strategies, utils
from tinygp import kernels


1.22.3
1.7.3


In [2]:
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
pylab.rcParams.update(params)

path = '/Users/chrislam/Desktop/gaspery/'

In [3]:
### Time to prep ### 

### observing parameters
sigma_ks = []
c = 1 # observing cadence of once per day
start = 2352
n_obs = 200

### target parameters
Mstar = 0.5 # Solar masses
sigma_wn_rv = 500 # cm/s (from Klein+ 2020)
p = 8.5 # orbital period, days
Prot = 4.86 # rotation period, days
K = 580 # cm/s
T0 = 2360 # arbitrarily chosen fiducial central transit time, in BJD

### correlated noise parameters, from Klein+ 2021 for AU Mic
Tau = 100/np.sqrt(2) # active region lifetime; days
eta = 0.4/np.sqrt(2) # 0.1, 0.3, 0.58, 0.9 # smoothing parameter
sigma_qp_rv = 47 * 1e2 # modified Jeffreys prior +11, -8 [cm/s]
sigma_wn_rv = 5 * 1e2 # [cm/s]

params = [Tau, eta, Prot, sigma_qp_rv, sigma_wn_rv]
theta = [K, p, T0]

As of this writing, it is currently 2/20/23, and the NOIRLab calendar for the 2023A semester (https://time-allocation.noirlab.edu/#/telescopes/classic-schedules/?telescopeId=3&semesterCode=2023A) says that the pre-booked nights on the telescope that we have to plan around are: 4/16-18, 4/20-21, the latter half of 4/22, 4/24-25, 5/13-15, 5/19-20, 6/18-19, the first half of 6/22, 6/24, the latter half of 6/25, 7/6-8, and 7/21. The calendar also says that the end of the observing semester is 7/23, which means any strategy must end by that day. 

Based on airmass.org or astroplan's determination of when in the year AU Mic is up at night, as well as the off nights prescribed above, let's start observing on 5/22/23. We will use 10am UTC as our default observation time (3am MST) because that's roughly when the star is highest at that time of year -- in reality, we would ask to be scheduled in the latter half of the night. If only the former half of a night has been blocked off, we still go ahead and observe that night because we assume our exposure is short enough. 

In [23]:
from astropy.time import Time

# start time and date
start = '2023-05-23T10:00:00'
start = Time(start, format='isot', scale='utc').jd

# construct off nights, bracketing a "night" as 8pm to 6am MST, or 3am to 1pm UTC the next day
# night is divided into halves around 1am local. 

offs = [ ['2023-06-19T03:00:00', '2023-06-19T13:00:00'], 
        ['2023-06-20T03:00:00', '2023-06-20T13:00:00'],
       ['2023-06-23T03:00:00', '2023-06-23T08:00:00'],
       ['2023-06-25T03:00:00', '2023-06-25T13:00:00'],
       ['2023-06-26T08:00:00', '2023-06-26T13:00:00'],
       ['2023-07-07T03:00:00', '2023-07-07T13:00:00'],
       ['2023-07-08T03:00:00', '2023-07-08T13:00:00'],
       ['2023-07-09T03:00:00', '2023-07-09T13:00:00'],
       ['2023-07-22T03:00:00', '2023-07-22T13:00:00']]

t = Time(offs, format='isot', scale='utc')
offs = t.jd
print("offs: ", juliant)

# stop observing because of end of season
stop = ['2023-07-24T03:00:00']

n_obs = 30

offs:  [[2460114.625      2460115.04166667]
 [2460115.625      2460116.04166667]
 [2460118.625      2460118.83333333]
 [2460120.625      2460121.04166667]
 [2460121.83333333 2460122.04166667]
 [2460132.625      2460133.04166667]
 [2460133.625      2460134.04166667]
 [2460134.625      2460135.04166667]
 [2460147.625      2460148.04166667]]


In [4]:
# offs: 4/16-18, 4/20-21, 4/22 latter half, 4/24-25, 5/13-15, 5/19-20, 6/18-19, 6/22 first half, 6/24, 6/25 latter half, 7/6-8, 7/21
# source: https://time-allocation.noirlab.edu/#/telescopes/classic-schedules/?year=2023&month=7&telescopeId=3&semesterCode=2023A
# I use 3am UTC bc that's 8pm MST and 1300 UTC bc that's 6am MST.
# remember we want 30-min exposures
# I use this BJD converter: https://www.aavso.org/jd-calculator

# this is assuming the conservative approach of skipping a day if we can't get the time
# for example, if the latter half is blocked off, we can still observe that day bc we do it at the start
# but if the first half is blocked off, while we can fit an observation at the end, we skip it still
offs = [2460050.62500, 2460051.62500, 2460052.62500, 2460054.62500, 2460055.62500, 2460058.62500, 2460059.62500, 
       2460077.62500, 2460078.62500, 2460079.62500, 2460083.62500, 2460084.62500,
       2460113.62500, 2460114.62500, 2460119.62500, 2460120.62500, 
       2460131.62500, 2460132.62500, 2460133.62500, 2460146.62500]

#start = 2460004.6250 # 8pm MST to UTC is 3am the next day, 3/31/23 03:00:00 UTC
start = 2460087.91667 # 3am MST to UTC is 10am the next day, 5/23/23 010:00:00 UTC

n_obs = 30 

Now, let's start prepping the parameters that describe our target. We assume a 30-Earth-mass planet orbiting an AU Mic-like star, using Table 3 from Klein+ 2020 as our correlated noise model hyperparameters. We also assume a central transit time of 2458623.5895 BJD, or May 20, 2019 at 02:08:52 UTC.

In [29]:
if ~((start > offs[0][0]) & (start < offs[0][1])):
    print("hi")

hi


In [31]:
m = 30 # Earth masses

# calculate RV based on 30 Earth-mass planet
K = utils.calculate_rv(Mstar, m, p) * 1e2 # cm/s

# central transit time
T0 = 2458342.22231 # BJD (latest from exoplanet.eu, Wittrock+ 2023)

### AU Mic properties
Tau = 100/np.sqrt(2) # 23.6 # days
eta = 0.4/np.sqrt(2) # 0.58
sigma_qp_rv = 4700 # modified Jeffreys prior +11, -8 [cm/s] # cm/s
sigma_wn_rv = 500 # cm/s 

params = [sigma_wn_rv, Tau, eta, Prot, sigma_qp_rv]
theta = [K, p, T0]

Use a quasi-periodic Gaussian Process kernel from tinygp to model the correlated noise of the star.

In [34]:
# build covariance matrix, characterized by a correlated noise model of the stellar signal
kernel = kernels.ExpSineSquared(scale=Prot, gamma=1/(2*eta**2)) # first term of exponential
kernel *= kernels.ExpSquared(scale=Tau) # other term of exponential
kernel *= sigma_qp_rv**2 # multiply by scalar

Time to cook

In [6]:
# instantiate Star object in order to feed covariance matrix with white/correlated noise
star = calculate_fi.Star(sigma_wn_rv = sigma_wn_rv, Tau = Tau, eta = eta, 
                         Prot = Prot, sigma_qp_rv = sigma_qp_rv)

# populate list of parameters to feed into cov_matrix_jax()
params = star.param_list()

# instantiate Planets object in order to feed Fisher Info calculation machinery
planet = calculate_fi.Planets(K = K, p = p, T0 = T0)

# populate list of parameters to feed into clam_jax_fim()
theta = planet.theta_list()

# instantiate Strategy object in order to build time series of observations
###strategy = strategies.Strategy(n_obs = n_obs, cadence = 1, start = start, offs=[], dropout=0.)

# build strategy aka time series of observations
###strat = strategy.gappy()

# build strategy time series  
strategy = strategies.Strategy(n_obs = n_obs, cadence = 1, start = start, offs=offs, dropout=0.)
strat = np.array(strategy.on_vs_off(on=on, off=off, twice_flag=True))

# if strategy is longer than baseline tolerance, don't bother with it
if strat[-1] - strat[0] > baseline_tol:
    sigma_ks[enum_on][enum_off] = np.nan
    fi_ks[enum_on][enum_off] = np.nan

else:
    # calculate covariance matrix
    sigma = star.cov_matrix_general(strat, kernel)

    # populate arguments for Fisher Info calculator
    args = np.array(strat), sigma, jnp.array(theta, dtype=float)

    # calculate FI
    fim = calculate_fi.clam_jax_fim(*args).block_until_ready()
    fi_ks[enum_on][enum_off] = fim[0][0]

    # invert FI matrix
    inv_fim = inv(fim)

    # top left element of matrix corresponds with RV semi-amplitude, K
    sigma_k = np.sqrt(inv_fim)[0][0]

    #print(f"sigma K for {m} Earth mass: ", sigma_k, " cm/s")
    #print(f"sigma K/K ratio for {m} Earth mass: ", sigma_k/K)

    sigma_ks[enum_on][enum_off] = sigma_k


NameError: name 'kernels' is not defined